<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [2]:

import json
import logging
import random
import time

import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [3]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [4]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [5]:
vocab_dict

'そんざい,ようそ,か,ち,りく,ドライブ,だんし,じまん,けっせき,にっこり,えんぎ,まずしい,たび,あたたまる,ふこう,バイオリン,うったえる,すすめる,じょうたつ,まざる,まじる,じょう,きゅう,うさぎ,ながす,こい,すくなくとも,あな,ふち,いわう,しょうめい,まつ,むしあつい,おせん,こしょう,かける,アイスクリーム,かさねる,まえもって,かんしゃ,ちかごろ,うらぎる,このむ,きよう,やとう,せいげん,そっくり,であう,かてい,くわえる,ささえる,おうじる,ぶつ,きょうきゅう,うけとる,じょうほう,ダウン,かっき,かいいん,いし,ねんじゅう,ブレーキ,きたい,だいめい,しゅうしょく,いっち,けんこう,きじ,ししゅつ,つよまる,どくしょ,さいしゅう,かゆい,あらし,するどい,とうちゃく,けむり,せいめい,そなえる,せいけつ,まつ,むすぶ,きょか,せいきゅう,なかみ,かみなり,きゅうりょう,はっけん,なかば,たとえ,かみのけ,ベルト,いつのまにか,ぎし,きみ,だまる,わける,いわば,がい,こうへい,すいみん,じゅん,たとえ,ねがい,ぶ,くだ,あんき,くだり,ふたたび,ホーム,かみ,たんい,だんたい,ながれ,おもいつく,こくせき,としつき,トラック,たび,ろんぶん,わく,おさえる,きずく,しつぼう,むく,よぶん,まねく,つよめる,よゆう,まよう,おたがい,ひはん,ねんかん,ながめる,じゅうみん,たにん,ぎもん,かんせい,ちじ,ゆかい,せいちょう,かなり,じたい,あいて,がくしゃ,いじ,ご,ぜいたく,ぜんこく,しまった (かん),つぐ,じつりょく,レベル,ふくろ,しゅうちゅう,りこん,こうえん,あかり,せいさん,にっこう,はた,どうじ,はっしゃ,くりかえす,げいじゅつ,なっとく,みりょく,よのなか,こむぎ,じょう,ふく,ごがく,だけど,ひげき,きこう,かた,ふつう,つきあい,いらいら,たいほ,おろす,デモ,そば,くろう,はたらき,ばくだい,へい,しかく,がくしゅう,まる,げんだい,くむ,いぜん,もと,かしこい,くべつ,つづき,ぼう,かる,ストレス,ず,おそわる,さくひん,およぎ,マーケット,もうしわけ,かんこう,きんこ,たんじょう,マッサージ,せいしん,きかい,はね,へいきん,うらやましい,しよう,へらす,しめる,せいき,おしゃべり,きぎょう,ちがいない,うつる,

#### load Models

In [6]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading): 8 questions
- このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese): 6 questions

Section 2: Reading Comprehension
- Information retrieval (info_retrieval): 0 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 0 questions

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The words in outline must be presented in Japanese only. The complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of moderate and very difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First, evenly pick Japanese words used in different part of sentence from 'Vocabulary' for questions in Vocabulary and Grammar, ensuring 40 percent of very difficult words. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [7]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [8]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [9]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-27 15:52:27,795 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [10]:
from IPython.display import display, Markdown

display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題用紙

## 第1部　語彙・文法

### のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)

8問　下線部の言葉の読み方を選ぶ問題です。40％は難しい語彙を含みます。

- **存在**
- **要素**
- **陸**
- **自慢**
- **欠席**
- **煙**
- **虚無**
- **哀れ**

### このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese)

6問　ひらがなで書かれた言葉を正しい漢字に直す問題です。40％は難しい語彙を含みます。

- **すすめる**
- **ふせぐ**
- **つよまる**
- **そんがい**
- **しつぼう**
- **まざる**

In [11]:
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = subsection['subsection_title'].split(' (')[1].rstrip(')')
        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 第1部　語彙・文法:   0%|          | 0/2 [00:00<?, ?it/s]
Processing のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading):   0%|          | 0/8 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-05-27 15:52:33,613 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 15:52:33,616 - INFO - Generated message: <li>
    この世界にはさまざまな生き物が<u>存在</u>している。
    <ol>
        <li>そんざい</li>
        <li>そんさい</li>
        <li>ぞんざい</li>
        <li>ぞんさい</li>
    </ol>
</li>


---REVISOR---


2025-05-27 15:52:38,307 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 15:52:38,309 - INFO - Refelect message: Thank you for your submission. Here is my detailed critique and recommendations based on your JLPT N3 level exam question:

1. Content Accuracy:
- The vocabulary word "存在" is appropriate for the JLPT N3 level, as it is a common noun and verb form that learners at this level should recognize.
- The sentence "この世界にはさまざまな生き物が存在している。" is clear, relevant, and appropriately challenging for N3 learners. It uses natural, everyday language and introduces a useful verb form.
- The four answer choices focus on the correct reading of the kanji "存在," which is a suitable task for testing vocabulary and kanji reading skills at N3.

2. Question and Answer Quality:
- The question is straightforward, asking for the correct reading of the underlined 

...

---REVISOR---


2025-05-27 15:57:07,539 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 15:57:07,542 - INFO - Refelect message: Thank you for your submission. Here is the detailed critique and recommendations based on the provided question:

1. Content Accuracy:
- The question tests the correct kanji for the verb "まざる" in the sentence "このジュースは水とよくまざるので、味がうすくなります."
- The sentence context is clear and appropriate for JLPT N3 level, as it involves understanding the meaning of "mix" or "blend" in a natural setting.
- The four options include:
  1. 混ざる (correct kanji for "to mix/blend")
  2. 交ざる (less common, but can mean "to mingle")
  3. 雑ざる (incorrect kanji usage)
  4. 減ざる (incorrect, not a valid verb)
- This selection effectively tests the learner's ability to distinguish correct kanji usage and meaning.

2. Question and Answer Quality:
- The question is cle

--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-05-27 15:57:08,482 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 15:57:08,485 - INFO - Formatted message: {'html_question': 'このジュースは水とよく<u>まざる</u>ので、味がうすくなります。', 'correct_answer': 1, 'choices': ['混ざる', '交ざる', '雑ざる', '減ざる']}

Processing このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese): 100%|██████████| 6/6 [02:07<00:00, 21.29s/it]
Processing 第1部　語彙・文法: 100%|██████████| 2/2 [04:40<00:00, 140.32s/it]

Total execution time: 280.66 seconds


In [12]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部　語彙・文法
  subsections:
  - subsection_title: のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)
    description: 8問　下線部の言葉の読み方を選ぶ問題です。40％は難しい語彙を含みます。
    question_topics:
    - topic: 存在
      result:
        html_question: この世界にはさまざまな生き物が<u>存在</u>している。
        correct_answer: 1
        choices:
        - そんざい
        - そんさい
        - ぞんざい
        - ぞんさい
    - topic: 要素
      result:
        html_question: このプロジェクトには大切な<u>要素</u>がいくつかあります。
        correct_answer: 2
        choices:
        - ようす
        - ようそ
        - ようしょ
        - ようせい
    - topic: ペットについて話す
      result:
        html_question: <ol><li>私は毎日<u>世話</u>をしている犬がいます。</li><li>この猫はとても<u>賢い</u>です。</li><li>うちの犬は<u>散歩</u>が好きです。</li><li>ペットの健康のために<u>注意</u>が必要です。</li><li>私の家には<u>小鳥</u>が三羽います。</li></ol>
        choices:
        - せわ
        - せか
        - よわ
        - よか
        - かしこい
        - けんこい
        - かんこい
        - けしかい
        - さんぼ
        - さんぽ
        - さぼ
        - さぽ
        - ちゅい
        - ちゅうい
        - しゅい
        - しゅうい
        - こちょう
        - ことり
        - しょうとり
        - しょうちょう
        correct_answer: 1
    - topic: 自慢
      result:
        html_question: 彼は自分の成績を<u>自慢</u>していました。
        correct_answer: 2
        choices:
        - じばん
        - じまん
        - じもん
        - じみん
    - topic: 欠席
      result:
        html_question: 体調が悪かったので、昨日の授業を<u>欠席</u>しました。
        correct_answer: 1
        choices:
        - けっせき
        - けっせつ
        - けっしゅう
        - けっしょく
    - topic: 交通手段について話す
      result:
        html_question: 私は毎朝<u>自転車</u>で駅まで行きます。
        correct_answer: 2
        choices:
        - じてんくるま
        - じてんしゃ
        - じどうしゃ
        - じしゃてん
    - topic: 虚無
      result:
        html_question: 彼は人生に<u>虚無</u>を感じているようだった。
        correct_answer: 1
        choices:
        - きょむ
        - きょぶ
        - こむ
        - けむ
    - topic: 哀れ
      result:
        html_question: その映画は多くの人に<u>哀れ</u>を感じさせた。
        correct_answer: 2
        choices:
        - あいれ
        - あわれ
        - あいわ
        - あわり
  - subsection_title: このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese)
    description: 6問　ひらがなで書かれた言葉を正しい漢字に直す問題です。40％は難しい語彙を含みます。
    question_topics:
    - topic: すすめる
      result:
        html_question: この本を友達に<u>すすめる</u>つもりです。
        choices:
        - 進める
        - 勧める
        - 薦める
        - 奨める
        correct_answer: 2
    - topic: ふせぐ
      result:
        html_question: 風邪を<u>ふせぐ</u>ために、手をよく洗いましょう。
        correct_answer: 1
        choices:
        - 防ぐ
        - 妨ぐ
        - 塞ぐ
        - 拒ぐ
    - topic: つよまる
      result:
        html_question: 台風の勢力が<u>つよまる</u>でしょう。
        choices:
        - 強増る
        - 強生る
        - 強まる
        - 強成る
        correct_answer: 3
    - topic: そんがい
      result:
        html_question: 台風で多くの家が<u>そんがい</u>を受けました。
        correct_answer: 1
        choices:
        - 損害
        - 存外
        - 尊敬
        - 損傷
    - topic: しつぼう
      result:
        html_question: 彼は結果を聞いて<u>しつぼう</u>したようだった。
        choices:
        - 失望
        - 失貌
        - 室望
        - 失防
        correct_answer: 1
    - topic: まざる
      result:
        html_question: このジュースは水とよく<u>まざる</u>ので、味がうすくなります。
        correct_answer: 1
        choices:
        - 混ざる
        - 交ざる
        - 雑ざる
        - 減ざる


In [13]:
output_data

{'sections': [{'section_title': '第1部\u3000語彙・文法',
   'subsections': [{'subsection_title': 'のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)',
     'description': '8問\u3000下線部の言葉の読み方を選ぶ問題です。40％は難しい語彙を含みます。',
     'question_topics': [{'topic': '存在',
       'result': {'html_question': 'この世界にはさまざまな生き物が<u>存在</u>している。',
        'correct_answer': 1,
        'choices': ['そんざい', 'そんさい', 'ぞんざい', 'ぞんさい']}},
      {'topic': '要素',
       'result': {'html_question': 'このプロジェクトには大切な<u>要素</u>がいくつかあります。',
        'correct_answer': 2,
        'choices': ['ようす', 'ようそ', 'ようしょ', 'ようせい']}},
      {'topic': 'ペットについて話す',
       'result': {'html_question': '<ol><li>私は毎日<u>世話</u>をしている犬がいます。</li><li>この猫はとても<u>賢い</u>です。</li><li>うちの犬は<u>散歩</u>が好きです。</li><li>ペットの健康のために<u>注意</u>が必要です。</li><li>私の家には<u>小鳥</u>が三羽います。</li></ol>',
        'choices': ['せわ',
         'せか',
         'よわ',
         'よか',
         'かしこい',
         'けんこい',
         'かんこい',
         'けしかい',
         'さんぼ',
         'さんぽ',
         'さぼ',
    

In [14]:
def render_to_html(sections):
    html = '<html><body style="text-align: left;">\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for idx, qtopic in enumerate(subsection.get("question_topics", []),1):
                # html += f'<h3>{qtopic.get("topic", "")}</h3>\n'
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += f'<p><strong>{idx} . {result["html_question"]}</strong></p>\n'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li>{idx}. <b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{idx}. {choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [15]:
html_output = render_to_html(output_data['sections'])
display(Markdown(html_output))

<html><body style="text-align: left;">
<h1>第1部　語彙・文法</h1>
<h2>のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading)</h2>
<p>8問　下線部の言葉の読み方を選ぶ問題です。40％は難しい語彙を含みます。</p>
<p><strong>1 . この世界にはさまざまな生き物が<u>存在</u>している。</strong></p>
<ul>
<li>1. <b>そんざい</b> <span style='color:green;'>(correct)</span></li>
<li>2. そんさい</li>
<li>3. ぞんざい</li>
<li>4. ぞんさい</li>
</ul>
<p><strong>2 . このプロジェクトには大切な<u>要素</u>がいくつかあります。</strong></p>
<ul>
<li>1. ようす</li>
<li>2. <b>ようそ</b> <span style='color:green;'>(correct)</span></li>
<li>3. ようしょ</li>
<li>4. ようせい</li>
</ul>
<p><strong>3 . <ol><li>私は毎日<u>世話</u>をしている犬がいます。</li><li>この猫はとても<u>賢い</u>です。</li><li>うちの犬は<u>散歩</u>が好きです。</li><li>ペットの健康のために<u>注意</u>が必要です。</li><li>私の家には<u>小鳥</u>が三羽います。</li></ol></strong></p>
<ul>
<li>1. <b>せわ</b> <span style='color:green;'>(correct)</span></li>
<li>2. せか</li>
<li>3. よわ</li>
<li>4. よか</li>
<li>5. かしこい</li>
<li>6. けんこい</li>
<li>7. かんこい</li>
<li>8. けしかい</li>
<li>9. さんぼ</li>
<li>10. さんぽ</li>
<li>11. さぼ</li>
<li>12. さぽ</li>
<li>13. ちゅい</li>
<li>14. ちゅうい</li>
<li>15. しゅい</li>
<li>16. しゅうい</li>
<li>17. こちょう</li>
<li>18. ことり</li>
<li>19. しょうとり</li>
<li>20. しょうちょう</li>
</ul>
<p><strong>4 . 彼は自分の成績を<u>自慢</u>していました。</strong></p>
<ul>
<li>1. じばん</li>
<li>2. <b>じまん</b> <span style='color:green;'>(correct)</span></li>
<li>3. じもん</li>
<li>4. じみん</li>
</ul>
<p><strong>5 . 体調が悪かったので、昨日の授業を<u>欠席</u>しました。</strong></p>
<ul>
<li>1. <b>けっせき</b> <span style='color:green;'>(correct)</span></li>
<li>2. けっせつ</li>
<li>3. けっしゅう</li>
<li>4. けっしょく</li>
</ul>
<p><strong>6 . 私は毎朝<u>自転車</u>で駅まで行きます。</strong></p>
<ul>
<li>1. じてんくるま</li>
<li>2. <b>じてんしゃ</b> <span style='color:green;'>(correct)</span></li>
<li>3. じどうしゃ</li>
<li>4. じしゃてん</li>
</ul>
<p><strong>7 . 彼は人生に<u>虚無</u>を感じているようだった。</strong></p>
<ul>
<li>1. <b>きょむ</b> <span style='color:green;'>(correct)</span></li>
<li>2. きょぶ</li>
<li>3. こむ</li>
<li>4. けむ</li>
</ul>
<p><strong>8 . その映画は多くの人に<u>哀れ</u>を感じさせた。</strong></p>
<ul>
<li>1. あいれ</li>
<li>2. <b>あわれ</b> <span style='color:green;'>(correct)</span></li>
<li>3. あいわ</li>
<li>4. あわり</li>
</ul>
<h2>このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese)</h2>
<p>6問　ひらがなで書かれた言葉を正しい漢字に直す問題です。40％は難しい語彙を含みます。</p>
<p><strong>1 . この本を友達に<u>すすめる</u>つもりです。</strong></p>
<ul>
<li>1. 進める</li>
<li>2. <b>勧める</b> <span style='color:green;'>(correct)</span></li>
<li>3. 薦める</li>
<li>4. 奨める</li>
</ul>
<p><strong>2 . 風邪を<u>ふせぐ</u>ために、手をよく洗いましょう。</strong></p>
<ul>
<li>1. <b>防ぐ</b> <span style='color:green;'>(correct)</span></li>
<li>2. 妨ぐ</li>
<li>3. 塞ぐ</li>
<li>4. 拒ぐ</li>
</ul>
<p><strong>3 . 台風の勢力が<u>つよまる</u>でしょう。</strong></p>
<ul>
<li>1. 強増る</li>
<li>2. 強生る</li>
<li>3. <b>強まる</b> <span style='color:green;'>(correct)</span></li>
<li>4. 強成る</li>
</ul>
<p><strong>4 . 台風で多くの家が<u>そんがい</u>を受けました。</strong></p>
<ul>
<li>1. <b>損害</b> <span style='color:green;'>(correct)</span></li>
<li>2. 存外</li>
<li>3. 尊敬</li>
<li>4. 損傷</li>
</ul>
<p><strong>5 . 彼は結果を聞いて<u>しつぼう</u>したようだった。</strong></p>
<ul>
<li>1. <b>失望</b> <span style='color:green;'>(correct)</span></li>
<li>2. 失貌</li>
<li>3. 室望</li>
<li>4. 失防</li>
</ul>
<p><strong>6 . このジュースは水とよく<u>まざる</u>ので、味がうすくなります。</strong></p>
<ul>
<li>1. <b>混ざる</b> <span style='color:green;'>(correct)</span></li>
<li>2. 交ざる</li>
<li>3. 雑ざる</li>
<li>4. 減ざる</li>
</ul>
</body></html>

In [16]:
# timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
# filename = f"./output/jlpt_simulator/JLPT_{timestamp}.html"

# with open(filename, "w", encoding="utf-8") as file:
#     file.write(html_output)